In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import pymc3 as pm
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import statistics
import warnings
import copy
from datetime import timedelta, datetime
import imageio
from matplotlib import cm
# import multiprocessing
import os
from pathlib import Path
import pydicom
import pytest
import scipy.ndimage as ndimage
from scipy.ndimage.interpolation import zoom
from skimage import measure, morphology, segmentation
from time import time, sleep
from tqdm import trange, tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, random_split, DistributedSampler, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

from scipy.stats import skew, kurtosis
from skimage.measure import label, regionprops
from skimage.segmentation import clear_border
import multiprocessing as mp
from functools import partial
import sys

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 176)
pd.set_option('display.max_columns', 35)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        break
#print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print(sys.version)

Autoencoder code:

In [ ]:
# !ls ../

In [ ]:
# root_dir = '/kaggle/input/osic-cached-dataset'
# train_dir = '/kaggle/input/osic-pulmonary-fibrosis-progression/train'
# test_dir = '/kaggle/input/osic-pulmonary-fibrosis-progression/test'
# model_file = '/kaggle/working/diophantus.pt'
# resize_dims = (40, 256, 256)
# clip_bounds = (-1000, 200)
# watershed_iterations = 1
# pre_calculated_mean = 0.02865046213070556
# latent_features = 2
# batch_size = 16
# learning_rate = 3e-5
# num_epochs = 10
# val_size = 0
# tensorboard_dir = '/kaggle/working/runs'

In [ ]:
# class CTScansDataset(Dataset):
#     def __init__(self, root_dir, transform=None):
#         self.root_dir = Path(root_dir)
#         self.patients = [p for p in self.root_dir.glob('*') if p.is_dir()]
#         self.transform = transform

#     def __len__(self):
#         return len(self.patients)

#     def __getitem__(self, idx):
#         if torch.is_tensor(idx):
#             idx = idx.tolist()

#         image, metadata = self.load_scan(self.patients[idx])
#         sample = {'image': image, 'metadata': metadata}
#         if self.transform:
#             sample = self.transform(sample)

#         return sample

#     def save(self, path):
#         t0 = time()
#         Path(path).mkdir(exist_ok=True, parents=True)
#         print('Saving pre-processed dataset to disk')
#         sleep(1)
#         cum = 0

#         bar = trange(len(self))
#         for i in bar:
#             sample = self[i]
#             image, data = sample['image'], sample['metadata']
#             cum += torch.mean(image).item()

#             bar.set_description(f'Saving CT scan {data.PatientID}')
#             fname = Path(path) / f'{data.PatientID}.pt'
#             torch.save(image, fname)

#         sleep(1)
#         bar.close()
#         print(f'Done! Time {timedelta(seconds=time() - t0)}\n'
#               f'Mean value: {cum / len(self)}')

#     def get_patient(self, patient_id):
#         patient_ids = [str(p.stem) for p in self.patients]
#         return self.__getitem__(patient_ids.index(patient_id))

#     @staticmethod
#     def load_scan(path):
#         slices = [pydicom.read_file(p) for p in path.glob('*.dcm')]
#         try:
#             slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
#         except AttributeError:
#             warnings.warn(f'Patient {slices[0].PatientID} CT scan does not '
#                           f'have "ImagePositionPatient". Assuming filenames '
#                           f'in the right scan order.')

#         image = np.stack([s.pixel_array.astype(float) for s in slices])
#         return image, slices[0]

# class CropBoundingBox:
#     @staticmethod
#     def bounding_box(img3d: np.array):
#         mid_img = img3d[int(img3d.shape[0] / 2)]
#         same_first_row = (mid_img[0, :] == mid_img[0, 0]).all()
#         same_first_col = (mid_img[:, 0] == mid_img[0, 0]).all()
#         if same_first_col and same_first_row:
#             return True
#         else:
#             return False

#     def __call__(self, sample):
#         image, data = sample['image'], sample['metadata']
#         if not self.bounding_box(image):
#             return sample

#         mid_img = image[int(image.shape[0] / 2)]
#         r_min, r_max = None, None
#         c_min, c_max = None, None
#         for row in range(mid_img.shape[0]):
#             if not (mid_img[row, :] == mid_img[0, 0]).all() and r_min is None:
#                 r_min = row
#             if (mid_img[row, :] == mid_img[0, 0]).all() and r_max is None \
#                     and r_min is not None:
#                 r_max = row
#                 break

#         for col in range(mid_img.shape[1]):
#             if not (mid_img[:, col] == mid_img[0, 0]).all() and c_min is None:
#                 c_min = col
#             if (mid_img[:, col] == mid_img[0, 0]).all() and c_max is None \
#                     and c_min is not None:
#                 c_max = col
#                 break

#         image = image[:, r_min:r_max, c_min:c_max]
#         return {'image': image, 'metadata': data}

# class ConvertToHU:
#     def __call__(self, sample):
#         image, data = sample['image'], sample['metadata']

#         img_type = data.ImageType
#         is_hu = img_type[0] == 'ORIGINAL' and not (img_type[2] == 'LOCALIZER')
#         # if not is_hu:
#         #     warnings.warn(f'Patient {data.PatientID} CT Scan not cannot be'
#         #                   f'converted to Hounsfield Units (HU).')

#         intercept = data.RescaleIntercept
#         slope = data.RescaleSlope
#         image = (image * slope + intercept).astype(np.int16)
#         return {'image': image, 'metadata': data}
    
# class Resize:
#     def __init__(self, output_size):
#         assert isinstance(output_size, tuple)
#         self.output_size = output_size

#     def __call__(self, sample):
#         image, data = sample['image'], sample['metadata']
#         resize_factor = np.array(self.output_size) / np.array(image.shape)
#         image = zoom(image, resize_factor, mode='nearest')
#         return {'image': image, 'metadata': data}
    
# class Clip:
#     def __init__(self, bounds=(-1000, 500)):
#         self.min = min(bounds)
#         self.max = max(bounds)

#     def __call__(self, sample):
#         image, data = sample['image'], sample['metadata']
#         image[image < self.min] = self.min
#         image[image > self.max] = self.max
#         return {'image': image, 'metadata': data}
    
# class MaskWatershed:
#     def __init__(self, min_hu, iterations, show_tqdm):
#         self.min_hu = min_hu
#         self.iterations = iterations
#         self.show_tqdm = show_tqdm

#     def __call__(self, sample):
#         image, data = sample['image'], sample['metadata']

#         stack = []
#         if self.show_tqdm:
#             bar = trange(image.shape[0])
#             bar.set_description(f'Masking CT scan {data.PatientID}')
#         else:
#             bar = range(image.shape[0])
#         for slice_idx in bar:
#             sliced = image[slice_idx]
#             stack.append(self.seperate_lungs(sliced, self.min_hu,
#                                              self.iterations))

#         return {
#             'image': np.stack(stack),
#             'metadata': sample['metadata']
#         }

#     @staticmethod
#     def seperate_lungs(image, min_hu, iterations):
#         h, w = image.shape[0], image.shape[1]

#         marker_internal, marker_external, marker_watershed = MaskWatershed.generate_markers(image)

#         # Sobel-Gradient
#         sobel_filtered_dx = ndimage.sobel(image, 1)
#         sobel_filtered_dy = ndimage.sobel(image, 0)
#         sobel_gradient = np.hypot(sobel_filtered_dx, sobel_filtered_dy)
#         sobel_gradient *= 255.0 / np.max(sobel_gradient)

#         watershed = morphology.watershed(sobel_gradient, marker_watershed)

#         outline = ndimage.morphological_gradient(watershed, size=(3,3))
#         outline = outline.astype(bool)

#         # Structuring element used for the filter
#         blackhat_struct = [[0, 0, 1, 1, 1, 0, 0],
#                            [0, 1, 1, 1, 1, 1, 0],
#                            [1, 1, 1, 1, 1, 1, 1],
#                            [1, 1, 1, 1, 1, 1, 1],
#                            [1, 1, 1, 1, 1, 1, 1],
#                            [0, 1, 1, 1, 1, 1, 0],
#                            [0, 0, 1, 1, 1, 0, 0]]

#         blackhat_struct = ndimage.iterate_structure(blackhat_struct, iterations)

#         # Perform Black Top-hat filter
#         outline += ndimage.black_tophat(outline, structure=blackhat_struct)

#         lungfilter = np.bitwise_or(marker_internal, outline)
#         lungfilter = ndimage.morphology.binary_closing(lungfilter, structure=np.ones((5,5)), iterations=3)

#         segmented = np.where(lungfilter == 1, image, min_hu * np.ones((h, w)))

#         return segmented  #, lungfilter, outline, watershed, sobel_gradient

#     @staticmethod
#     def generate_markers(image, threshold=-400):
#         h, w = image.shape[0], image.shape[1]

#         marker_internal = image < threshold
#         marker_internal = segmentation.clear_border(marker_internal)
#         marker_internal_labels = measure.label(marker_internal)

#         areas = [r.area for r in measure.regionprops(marker_internal_labels)]
#         areas.sort()

#         if len(areas) > 2:
#             for region in measure.regionprops(marker_internal_labels):
#                 if region.area < areas[-2]:
#                     for coordinates in region.coords:
#                         marker_internal_labels[coordinates[0], coordinates[1]] = 0

#         marker_internal = marker_internal_labels > 0

#         # Creation of the External Marker
#         external_a = ndimage.binary_dilation(marker_internal, iterations=10)
#         external_b = ndimage.binary_dilation(marker_internal, iterations=55)
#         marker_external = external_b ^ external_a

#         # Creation of the Watershed Marker
#         marker_watershed = np.zeros((h, w), dtype=np.int)
#         marker_watershed += marker_internal * 255
#         marker_watershed += marker_external * 128

#         return marker_internal, marker_external, marker_watershed
    
# class Normalize:
#     def __init__(self, bounds=(-1000, 500)):
#         self.min = min(bounds)
#         self.max = max(bounds)

#     def __call__(self, sample):
#         image, data = sample['image'], sample['metadata']
#         image = image.astype(np.float)
#         image = (image - self.min) / (self.max - self.min)
#         return {'image': image, 'metadata': data}
    

# class ToTensor:
#     def __init__(self, add_channel=True):
#         self.add_channel = add_channel

#     def __call__(self, sample):
#         image, data = sample['image'], sample['metadata']
#         if self.add_channel:
#             image = np.expand_dims(image, axis=0)

#         return {'image': torch.from_numpy(image), 'metadata': data}
    
    
# class ZeroCenter:
#     def __init__(self, pre_calculated_mean):
#         self.pre_calculated_mean = pre_calculated_mean

#     def __call__(self, tensor):
#         return tensor - self.pre_calculated_mean


In [ ]:
# def show(list_imgs, cmap=cm.bone):
#     list_slices = []
#     for img3d in list_imgs:
#         slc = int(img3d.shape[0] / 2)
#         img = img3d[slc]
#         list_slices.append(img)
    
#     fig, axs = plt.subplots(1, 5, figsize=(15, 7))
#     for i, img in enumerate(list_slices):
#         axs[i].imshow(img, cmap=cmap)
#         axs[i].axis('off')
        
#     plt.show()

In [ ]:
# # Show some images
# test_img = CTScansDataset(
#     root_dir=test_dir,
#     transform=transforms.Compose([
#         CropBoundingBox(),
#         ConvertToHU(),
#         Resize(resize_dims),
#         Clip(bounds=clip_bounds),
#         MaskWatershed(min_hu=min(clip_bounds), iterations=1, show_tqdm=True),
#         Normalize(bounds=clip_bounds)
#     ]))

# list_imgs = [test_img[i]['image'] for i in range(len(test_img))]
# show(list_imgs)

In [ ]:
# Only uncomment if pre-processing train data from scratch.
# train_img = CTScansDataset(
#     root_dir=train_dir,
#     transform=transforms.Compose([
#         CropBoundingBox(),
#         ConvertToHU(),
#         Resize(resize_dims),
#         Clip(bounds=clip_bounds),
#         MaskWatershed(min_hu=min(clip_bounds), iterations=1, show_tqdm=True),
#         Normalize(bounds=clip_bounds)
#     ]))

# list_imgs = [train_img[i]['image'] for i in range(len(train_img))]
# show(list_imgs)

In [ ]:
# class CTTensorsDataset(Dataset):
#     def __init__(self, root_dir, transform=None):
#         self.root_dir = Path(root_dir)
#         self.tensor_files = sorted([f for f in self.root_dir.glob('*.pt')])
#         self.transform = transform

#     def __len__(self):
#         return len(self.tensor_files)

#     def __getitem__(self, item):
#         if torch.is_tensor(item):
#             item = item.tolist()

#         image = torch.load(self.tensor_files[item])
#         if self.transform:
#             image = self.transform(image)

#         return {
#             'patient_id': self.tensor_files[item].stem,
#             'image': image
#         }

#     def mean(self):
#         cum = 0
#         for i in range(len(self)):
#             sample = self[i]['image']
#             cum += torch.mean(sample).item()

#         return cum / len(self)

#     def random_split(self, val_size: float):
#         num_val = int(val_size * len(self))
#         num_train = len(self) - num_val
#         return random_split(self, [num_train, num_val])

In [ ]:
# torch.mean(data[0]['image']).item()

In [ ]:
# train_img = CTTensorsDataset(
#     root_dir=root_dir,
#     transform=ZeroCenter(pre_calculated_mean=pre_calculated_mean)
# )
# cum = 0
# for i in range(len(train_img)):
#     sample = train_img[i]['image']
#     cum += torch.mean(sample).item()

# assert cum / len(train_img) == pytest.approx(0)

In [ ]:
# class AutoEncoder(nn.Module):
#     def __init__(self, latent_features=latent_features):
#         super(AutoEncoder, self).__init__()
#         # Encoder
#         self.conv1 = nn.Conv3d(1, 16, 3)
#         self.conv2 = nn.Conv3d(16, 32, 3)
#         self.conv3 = nn.Conv3d(32, 96, 2)
#         self.conv4 = nn.Conv3d(96, 1, 1)
#         self.pool1 = nn.MaxPool3d(kernel_size=2, stride=2, return_indices=True)
#         self.pool2 = nn.MaxPool3d(kernel_size=3, stride=3, return_indices=True)
#         self.pool3 = nn.MaxPool3d(kernel_size=2, stride=2, return_indices=True)
#         self.pool4 = nn.MaxPool3d(kernel_size=2, stride=2, return_indices=True)
#         self.fc1 = nn.Linear(10 * 10, latent_features)
#         # Decoder
#         self.fc2 = nn.Linear(latent_features, 10 * 10)
#         self.deconv0 = nn.ConvTranspose3d(1, 96, 1)
#         self.deconv1 = nn.ConvTranspose3d(96, 32, 2)
#         self.deconv2 = nn.ConvTranspose3d(32, 16, 3)
#         self.deconv3 = nn.ConvTranspose3d(16, 1, 3)
#         self.unpool0 = nn.MaxUnpool3d(kernel_size=2, stride=2)
#         self.unpool1 = nn.MaxUnpool3d(kernel_size=2, stride=2)
#         self.unpool2 = nn.MaxUnpool3d(kernel_size=3, stride=3)
#         self.unpool3 = nn.MaxUnpool3d(kernel_size=2, stride=2)

#     def encode(self, x, return_partials=True):
#         # Encoder
#         x = self.conv1(x)
#         up3out_shape = x.shape
#         x, i1 = self.pool1(x)

#         x = self.conv2(x)
#         up2out_shape = x.shape
#         x, i2 = self.pool2(x)

#         x = self.conv3(x)
#         up1out_shape = x.shape
#         x, i3 = self.pool3(x)

#         x = self.conv4(x)
#         up0out_shape = x.shape
#         x, i4 = self.pool4(x)

#         x = x.view(-1, 10 * 10)
#         x = F.relu(self.fc1(x))

#         if return_partials:
#             return x, up3out_shape, i1, up2out_shape, i2, up1out_shape, i3, \
#                    up0out_shape, i4

#         else:
#             return x

#     def forward(self, x):
#         x, up3out_shape, i1, up2out_shape, i2, \
#         up1out_shape, i3, up0out_shape, i4 = self.encode(x)

#         # Decoder
#         x = F.relu(self.fc2(x))
#         x = x.view(-1, 1, 1, 10, 10)
#         x = self.unpool0(x, output_size=up0out_shape, indices=i4)
#         x = self.deconv0(x)
#         x = self.unpool1(x, output_size=up1out_shape, indices=i3)
#         x = self.deconv1(x)
#         x = self.unpool2(x, output_size=up2out_shape, indices=i2)
#         x = self.deconv2(x)
#         x = self.unpool3(x, output_size=up3out_shape, indices=i1)
#         x = self.deconv3(x)

#         return x

In [ ]:
# t0 = time()

# # Load the data
# data = CTTensorsDataset(
#     root_dir=root_dir,
#     transform=ZeroCenter(pre_calculated_mean=pre_calculated_mean)
# )
# # train_set, val_set = data.random_split(val_size)
# train_set = data
# val_set = None
# # datasets = {'train': train_set, 'val': val_set}
# datasets = {'train': train_set}
# dataloaders = {
#     x: DataLoader(
#         datasets[x],
#         batch_size=batch_size,
# #         shuffle=(x == 'train'),
#         shuffle=(False),
#         num_workers=2
# #     ) for x in ['train', 'val']}
#     ) for x in ['train']}

# # dataset_sizes = {x: len(datasets[x]) for x in ['train', 'val']}
# dataset_sizes = {x: len(datasets[x]) for x in ['train']}

# # Prepare for training
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = AutoEncoder(latent_features=latent_features).to(device)
# criterion = torch.nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# best_model_wts = None
# best_loss = np.inf

# date_time = datetime.now().strftime("%Y%m%d-%H%M")
# log_dir = Path(tensorboard_dir) / f'{date_time}'
# writer = SummaryWriter(log_dir)

In [ ]:
# # Training loop
# for epoch in range(num_epochs):

#     # Each epoch has a training and validation phase
# #     for phase in ['train', 'val']:
#     for phase in ['train']:
#         if phase == 'train':
#             model.train()  # Set model to training mode
#         else:
#             model.eval()   # Set model to evaluate mode

#         running_loss = 0.0
#         running_preds = 0

#         # Iterate over data.
#         bar = tqdm(dataloaders[phase])
#         for inputs in bar:
#             bar.set_description(f'Epoch {epoch + 1} {phase}'.ljust(20))
#             inputs = inputs['image'].to(device, dtype=torch.float)

#             # zero the parameter gradients
#             optimizer.zero_grad()

#             # forward
#             # track history if only in train
#             with torch.set_grad_enabled(phase == 'train'):
#                 outputs = model(inputs)
#                 loss = criterion(outputs, inputs)

#                 # backward + optimize only if in training phase
#                 if phase == 'train':
#                     loss.backward()
#                     optimizer.step()

#             # statistics
#             running_loss += loss.item() * inputs.size(0)
#             running_preds += inputs.size(0)
#             bar.set_postfix(loss=f'{running_loss / running_preds:0.6f}')

#         epoch_loss = running_loss / dataset_sizes[phase]
#         writer.add_scalar(f'Loss/{phase}', epoch_loss, epoch)

#         # deep copy the model
# #         if phase == 'val' and epoch_loss < best_loss:
#         if phase == 'train' and epoch_loss < best_loss:
#             best_loss = epoch_loss
#             best_model_wts = copy.deepcopy(model.state_dict())
#             torch.save(best_model_wts, model_file)

# # load best model weights
# model.load_state_dict(best_model_wts)

# print(f'Done! Time {timedelta(seconds=time() - t0)}')

In [ ]:
excluded_patients = ['ID00011637202177653955184', 'ID00052637202186188008618', 'ID00102637202206574119190']

In [ ]:
# slc = 0.5
# sample_id = np.random.randint(len(data))
# print(f'Inspecting CT Scan {data[sample_id]["patient_id"]}')

# fig, axs = plt.subplots(1, 2, figsize=(10, 7))

# sample = data[sample_id]['image'].squeeze(0).numpy()
# axs[0].imshow(sample[int(40 * slc), :, :], cmap=cm.bone)
# axs[0].axis('off')
# imageio.mimsave("sample_input.gif", sample, duration=0.1)

# with torch.no_grad():
#     img = data[sample_id]['image'].unsqueeze(0).float().to(device)
#     latent_features = model.encode(img, return_partials=False)\
#         .squeeze().cpu().numpy().tolist()
#     outputs = model(img).squeeze().cpu().numpy()

# axs[1].imshow(outputs[int(40 * slc), :, :], cmap=cm.bone)
# axs[1].axis('off')

# imageio.mimsave("sample_output.gif", outputs, duration=0.1)

# rmse = ((sample - outputs)**2).mean()
# plt.show()
# print(f'Latent features: {latent_features} \nLoss: {rmse}')

CT Scan preprocessing and feature generation code:

In [ ]:
class CTScansDataset(Dataset):
    def __init__(self, root_dir, transform=None, excl_patients=None):
        self.root_dir = Path(root_dir)
        self.patients = [p for p in self.root_dir.glob('*') if (p.is_dir() and p.name not in excl_patients)]
        self.transform = transform
        self.excl_patients = excl_patients

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image, metadata, msbs = self.load_scan(self.patients[idx])
        sample = {'image': image, 'metadata': metadata, 'msbs': msbs}
        if self.transform:
            sample = self.transform(sample)

        return sample

    def save(self, path):
        t0 = time()
        Path(path).mkdir(exist_ok=True, parents=True)
        print('Saving pre-processed dataset to disk')
        sleep(1)
        cum = 0
        var = 0

        bar = trange(len(self))
        for i in bar:
            sample = self[i]
            image, data = sample['image'], sample['metadata']
            cum += torch.mean(image).item()
            var += torch.var(image).item()

            bar.set_description(f'Saving CT scan {data.PatientID}')
            fname = Path(path) / f'{data.PatientID}.pt'
            torch.save(image, fname)

        sleep(1)
        bar.close()
        print(f'Done! Time {timedelta(seconds=time() - t0)}\n'
              f'Mean value: {cum / len(self)}\n'
              f'Std value: {(var / len(self)) ** (1/2)}')

    def get_patient(self, patient_id):
        patient_ids = [str(p.stem) for p in self.patients]
        return self.__getitem__(patient_ids.index(patient_id))

    @staticmethod
    def load_scan(path):
        raw_slices = [pydicom.read_file(p) for p in path.glob('*.dcm')]
        final_slices = [s for s in raw_slices
                        if hasattr(s, 'ImagePositionPatient')]

        if len(final_slices) > 0:
            final_slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
            msbs = 0
            for i in range(len(final_slices) - 1):
                a = final_slices[i].ImagePositionPatient[2]
                b = final_slices[i + 1].ImagePositionPatient[2]
                msbs += abs(a - b)

            msbs /= len(final_slices)
            image = np.stack([s.pixel_array.astype(float) for s in final_slices])
            metadata = final_slices[0]

        else:
            # Guess based on eyballing other mean spaces between slices:
            msbs = None
            if raw_slices[0].PatientID == 'ID00132637202222178761324':
                msbs = 0.7
            elif raw_slices[0].PatientID == 'ID00128637202219474716089':
                msbs = 5.

            warnings.warn(f'Patient {raw_slices[0].PatientID} CT scan does '
                          f'not have "ImagePositionPatient". Assuming '
                          f'filenames in the right scan order. Also, assuming'
                          f'mean space between slices of {msbs}')

            image = np.stack([s.pixel_array.astype(float) for s in raw_slices])
            metadata = raw_slices[0]

        return image, metadata, msbs

In [ ]:
dataset = CTScansDataset(root_dir='../input/osic-pulmonary-fibrosis-progression/train', excl_patients=excluded_patients)
dataset.patients

In [ ]:
class CropBoundingBox:
    @staticmethod
    def bounding_box(img3d: np.array):
        mid_img = img3d[int(img3d.shape[0] / 2)]
        same_first_row = (mid_img[0, :] == mid_img[0, 0]).all()
        same_first_col = (mid_img[:, 0] == mid_img[0, 0]).all()
        if same_first_col and same_first_row:
            return True
        else:
            return False

    def __call__(self, sample):
        image, data = sample['image'], sample['metadata']
        if not self.bounding_box(image):
            return sample

        mid_img = image[int(image.shape[0] / 2)]
        r_min, r_max = None, None
        c_min, c_max = None, None
        for row in range(mid_img.shape[0]):
            if not (mid_img[row, :] == mid_img[0, 0]).all() and r_min is None:
                r_min = row
            if (mid_img[row, :] == mid_img[0, 0]).all() and r_max is None \
                    and r_min is not None:
                r_max = row
                break

        for col in range(mid_img.shape[1]):
            if not (mid_img[:, col] == mid_img[0, 0]).all() and c_min is None:
                c_min = col
            if (mid_img[:, col] == mid_img[0, 0]).all() and c_max is None \
                    and c_min is not None:
                c_max = col
                break

        image = image[:, r_min:r_max, c_min:c_max]
        return {'image': image, 'metadata': data, 'msbs': sample['msbs']}

In [ ]:
class ConvertToHU:
    def __call__(self, sample):
        image, data = sample['image'], sample['metadata']

        img_type = data.ImageType
        is_hu = img_type[0] == 'ORIGINAL' and not (img_type[2] == 'LOCALIZER')
        # if not is_hu:
        #     warnings.warn(f'Patient {data.PatientID} CT Scan not cannot be'
        #                   f'converted to Hounsfield Units (HU).')

        intercept = data.RescaleIntercept
        slope = data.RescaleSlope
        image = (image * slope + intercept).astype(np.int16)
        return {'image': image, 'metadata': data, 'msbs': sample['msbs']}

In [ ]:
class Resample:
    def __init__(self, new_spacing=(1, 1, 1)):
        """new_spacing means now much every pixel represent in mm, in each
        dimension. E.g. 2, 2, 2 means every pixel represent 2mm in  every
        dimension.
        """
        assert isinstance(new_spacing, tuple)
        self.new_spacing = new_spacing

    def __call__(self, sample):
        image, data, msbs = sample['image'], sample['metadata'], sample['msbs']

        spacing = np.array([msbs] + list(data.PixelSpacing), dtype=np.float32)
        resize_factor = spacing / self.new_spacing
        new_real_shape = image.shape * resize_factor
        new_shape = np.round(new_real_shape)
        real_resize_factor = new_shape / image.shape

        image = zoom(image, real_resize_factor, mode='nearest')
        return {'image': image, 'metadata': data}

In [ ]:
class Clip:
    def __init__(self, bounds=(-1000, 500)):
        self.min = min(bounds)
        self.max = max(bounds)

    def __call__(self, sample):
        image, data = sample['image'], sample['metadata']
        image[image < self.min] = self.min
        image[image > self.max] = self.max
        return {'image': image, 'metadata': data}

In [ ]:
class MaskWatershed:
    def __init__(self, min_hu, iterations, show_tqdm):
        self.min_hu = min_hu
        self.iterations = iterations
        self.show_tqdm = show_tqdm

    def __call__(self, sample):
        image, data = sample['image'], sample['metadata']

        stack = []
        if self.show_tqdm:
            bar = trange(image.shape[0])
            bar.set_description(f'Masking CT scan {data.PatientID}')
        else:
            bar = range(image.shape[0])
        for slice_idx in bar:
            sliced = image[slice_idx]
            stack.append(self.seperate_lungs(sliced, self.min_hu,
                                             self.iterations))

        return {
            'image': np.stack(stack),
            'metadata': sample['metadata']
        }

    @staticmethod
    def seperate_lungs(image, min_hu, iterations):
        h, w = image.shape[0], image.shape[1]

        marker_internal, marker_external, marker_watershed = MaskWatershed.generate_markers(image)

        # Sobel-Gradient
        sobel_filtered_dx = ndimage.sobel(image, 1)
        sobel_filtered_dy = ndimage.sobel(image, 0)
        sobel_gradient = np.hypot(sobel_filtered_dx, sobel_filtered_dy)
        sobel_gradient *= 255.0 / np.max(sobel_gradient)

        watershed = morphology.watershed(sobel_gradient, marker_watershed)

        outline = ndimage.morphological_gradient(watershed, size=(3,3))
        outline = outline.astype(bool)

        # Structuring element used for the filter
        blackhat_struct = [[0, 0, 1, 1, 1, 0, 0],
                           [0, 1, 1, 1, 1, 1, 0],
                           [1, 1, 1, 1, 1, 1, 1],
                           [1, 1, 1, 1, 1, 1, 1],
                           [1, 1, 1, 1, 1, 1, 1],
                           [0, 1, 1, 1, 1, 1, 0],
                           [0, 0, 1, 1, 1, 0, 0]]

        blackhat_struct = ndimage.iterate_structure(blackhat_struct, iterations)

        # Perform Black Top-hat filter
        outline += ndimage.black_tophat(outline, structure=blackhat_struct)

        lungfilter = np.bitwise_or(marker_internal, outline)
        lungfilter = ndimage.morphology.binary_closing(lungfilter, structure=np.ones((5,5)), iterations=3)

        segmented = np.where(lungfilter == 1, image, min_hu * np.ones((h, w)))

        return segmented

    @staticmethod
    def generate_markers(image, threshold=-400):
        h, w = image.shape[0], image.shape[1]

        marker_internal = image < threshold
        marker_internal = segmentation.clear_border(marker_internal)
        marker_internal_labels = measure.label(marker_internal)

        areas = [r.area for r in measure.regionprops(marker_internal_labels)]
        areas.sort()

        if len(areas) > 2:
            for region in measure.regionprops(marker_internal_labels):
                if region.area < areas[-2]:
                    for coordinates in region.coords:
                        marker_internal_labels[coordinates[0], coordinates[1]] = 0

        marker_internal = marker_internal_labels > 0

        # Creation of the External Marker
        external_a = ndimage.binary_dilation(marker_internal, iterations=10)
        external_b = ndimage.binary_dilation(marker_internal, iterations=55)
        marker_external = external_b ^ external_a

        # Creation of the Watershed Marker
        marker_watershed = np.zeros((h, w), dtype=np.int)
        marker_watershed += marker_internal * 255
        marker_watershed += marker_external * 128

        return marker_internal, marker_external, marker_watershed

In [ ]:
class LungVolume:
    def __init__(self, unit, spacing, min_HU):
        self.unit = unit
        self.spacing = spacing
        self.min_HU = min_HU

    def __call__(self, sample):
        image = sample['image']

        volume = (image > self.min_HU).sum()
        volumes_in_perc = volume / np.prod(image.shape)
        volumes_in_litr = volumes_in_perc * np.prod(image.shape) * \
                          np.prod(self.spacing) / 10**6

        if self.unit == '%':
            sample['volume'] = volumes_in_perc
        elif self.unit == 'l':
            sample['volume'] = volumes_in_litr

        return sample

In [ ]:
class ChestCircumference:
    def __init__(self, spacing, min_HU):
        self.spacing = spacing
        self.min_HU = min_HU

    def ellipsis_perimeter(self, img):
        # trim slice, removing zero rows and cols
        img = img[~(img == self.min_HU).all(1)].T
        img = img[~(img == self.min_HU).all(1)]
        # ellipsis perimeter approximation 2
        # https://www.mathsisfun.com/geometry/ellipse-perimeter.html
        a, b = img.shape
        a /= 2
        b /= 2
        p = np.pi * (3 * (a + b) - np.sqrt((3 * a + b) * (a + 3 * b)))
        return p

    def __call__(self, sample):
        image = sample['image']

        perimeters = []
        for slice_idx in range(image.shape[0]):
            slc = image[slice_idx]
            if slc.max() != self.min_HU:  # There's something in the slice
                perimeters.append(self.ellipsis_perimeter(slc))
            else:
                perimeters.append(0)

        # Transform in mm
        cc_in_mm = max(perimeters) * self.spacing[1] * self.spacing[2]
        sample['chest'] = int(cc_in_mm)
        return sample

In [ ]:
class LungHeight:
    def __init__(self, spacing):
        self.spacing = spacing

    def __call__(self, sample):
        image = sample['image']
        sample['height'] = image.shape[0] * self.spacing[0]
        return sample

In [ ]:
class HUHistogram:
    def __init__(self, bounds, bins):
        self.bounds = bounds
        self.bins = bins

    def __call__(self, sample):
        image = sample['image']
        pixels = image[image != min(self.bounds)]

        if pixels.shape[0] > 0:
            sample['mean'] = pixels.mean()
            sample['std'] = pixels.std()
            sample['skew'] = skew(pixels)
            sample['kurtosis'] = kurtosis(pixels)

            hist, _ = np.histogram(pixels, bins=self.bins, range=self.bounds)
            sample['hist'] = hist / pixels.size  # Normalized
            # Density = True in np.histogram doesn't work as expected
        else:
            sample['mean'] = 0
            sample['std'] = 0
            sample['skew'] = 0
            sample['kurtosis'] = 0
            sample['hist'] = np.zeros(self.bins)

        return sample

In [ ]:
def gennerate_row(dataset, i):
    sample = dataset[i]
    print(sample['metadata'].PatientID)
    df = pd.DataFrame(columns=np.arange(8).tolist())
    df.loc[0] = [sample['metadata'].PatientID, sample["volume"],
                 sample["chest"], sample["height"], sample["mean"],
                 sample["std"], sample["skew"], sample["kurtosis"]]

    dfhist = pd.DataFrame()
    dfhist[0] = sample['hist']
    dfhist = dfhist.T
    df = pd.concat([df, dfhist], axis=1, ignore_index=True)

    columns = ['Patient', 'Volume', 'Chest', 'Height', 'Mean',
               'Std', 'Skew', 'Kurtosis']
    columns += [f'bin{b + 1}' for b in range(len(dfhist.columns))]
    df.columns = columns
    return df

In [ ]:
t0 = time()
dataset = CTScansDataset(
    root_dir='../input/osic-pulmonary-fibrosis-progression/train',
    transform=transforms.Compose([
        CropBoundingBox(),
        ConvertToHU(),
        Resample(new_spacing=(1, 1, 1)),
        Clip(bounds=(-1000, 400)),
        MaskWatershed(min_hu=-1000, iterations=1, show_tqdm=False),
        # Features
        LungVolume(unit='l',
                   spacing=(1, 1, 1),
                   min_HU=-1000),
        ChestCircumference(spacing=(1, 1, 1), min_HU=-1000),
        LungHeight(spacing=(1, 1, 1)),
        HUHistogram(bounds=(-1000, 400), bins=14)
    ]),
    excl_patients=excluded_patients
)

f = partial(gennerate_row, dataset)
with mp.Pool(processes=mp.cpu_count()) as pool:
    results = list(tqdm(pool.imap_unordered(f, range(len(dataset))),
                        total=len(dataset)))

features_train = pd.concat(results, ignore_index=True)
features_train.to_csv('features_train.csv', index=False)

print(f'\nDone! Time: {timedelta(seconds=time() - t0)}')

In [ ]:
features_train.head()

In [ ]:
t0 = time()
dataset = CTScansDataset(
    root_dir='../input/osic-pulmonary-fibrosis-progression/test',
    transform=transforms.Compose([
        CropBoundingBox(),
        ConvertToHU(),
        Resample(new_spacing=(1, 1, 1)),
        Clip(bounds=(-1000, 400)),
        MaskWatershed(min_hu=-1000, iterations=1, show_tqdm=False),
        # Features
        LungVolume(unit='l',
                   spacing=(1, 1, 1),
                   min_HU=-1000),
        ChestCircumference(spacing=(1, 1, 1), min_HU=-1000),
        LungHeight(spacing=(1, 1, 1)),
        HUHistogram(bounds=(-1000, 400), bins=14)
    ]))

f = partial(gennerate_row, dataset)
with mp.Pool(processes=mp.cpu_count()) as pool:
    results = list(tqdm(pool.imap_unordered(f, range(len(dataset))),
                        total=len(dataset)))

features_test = pd.concat(results, ignore_index=True)
features_test.to_csv('features_test.csv', index=False)

print(f'\nDone! Time: {timedelta(seconds=time() - t0)}')

In [ ]:
features_test.head()

In [ ]:
features = pd.concat([features_train, features_test])
features.head(176)

Bayesian Code:

In [ ]:
# Preprocessing
# train is resused/editted. train_raw is left alone
train = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
train_raw = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
test = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')

# train.drop(train[train.Patient == 'ID00197637202246865691526'].index, inplace=True)

# exclude_test_patient_data_from_trainset = False

# if exclude_test_patient_data_from_trainset:
#     train = train[~train['Patient'].isin(test['Patient'].unique())]

train = pd.concat([train, test], axis=0, ignore_index=True)\
    .drop_duplicates()
le_id = LabelEncoder()
train['PatientID'] = le_id.fit_transform(train['Patient'])

train.head()

In [ ]:
def add_baselines(data):
    aux = data[['Patient', 'Weeks']].groupby('Patient')\
        .min().reset_index()
    aux = pd.merge(aux, data[['Patient', 'Weeks', 'FVC', 'Percent']], how='left', 
                   on=['Patient', 'Weeks'])
    aux = aux.groupby('Patient').mean().reset_index()
    aux['Weeks'] = aux['Weeks'].astype(int)
    aux['FVC'] = aux['FVC'].astype(int)
    aux['Percent'] = aux['Percent'].astype(float)
    data = pd.merge(data, aux, how='left', on='Patient', suffixes=('', '_base'))
    return data

train = add_baselines(train)
test = add_baselines(test)
train.head()

In [ ]:
def patient_class(row):
    if row['Sex'] == 'Male':
        if row['SmokingStatus'] == 'Currently smokes':
            return 0
        elif row['SmokingStatus'] == 'Ex-smoker':
            return 1
        elif row['SmokingStatus'] == 'Never smoked':
            return 2
    else:
        if row['SmokingStatus'] == 'Currently smokes':
            return 3
        elif row['SmokingStatus'] == 'Ex-smoker':
            return 4
        elif row['SmokingStatus'] == 'Never smoked':
            return 5

train['Class'] = train.apply(patient_class, axis=1)
test['Class'] = test.apply(patient_class, axis=1)
test.head()
train.loc[train['Patient']=='ID00007637202177411956430']['Class'].max()

In [ ]:
patients = pd.DataFrame()
# Very simple pre-processing: creating patient indexes


patients = train[['Patient', 'PatientID', 'Age', 'Class', 'Weeks_base', 'FVC_base', 'Percent_base']].drop_duplicates()
fvc_data = train[['Patient', 'PatientID', 'Weeks', 'FVC']]
patients['FVC_nom_base'] = patients['FVC_base']*(100./patients['Percent_base'])
patients.head()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
features.head()

In [ ]:
train = train.merge(features, on=['Patient'])
train.head()

In [ ]:
test = test.merge(features, on=['Patient'])
test.head()

In [ ]:
patients = patients.merge(features, on=['Patient'])
patients.head()

In [ ]:
patient_ids = patients['Patient']
#patient_ids

for patient_id in patient_ids:
    df_patient = pd.DataFrame(train.loc[train.Patient == patient_id])
    x = np.array(train.loc[train.Patient == patient_id, 'Weeks'].values).reshape(-1,1)
    y = train.loc[train.Patient == patient_id, 'FVC'].values
    regr = linear_model.LinearRegression()
    regr.fit(x,y)
    patients.loc[patients.Patient==patient_id,'gradient'] = regr.coef_[0]
    patients.loc[patients.Patient==patient_id,'intercept'] = regr.intercept_
    patients.loc[patients.Patient==patient_id,'intercept'] = regr.intercept_
    y_pred = regr.predict(x)
    patients.loc[patients.Patient==patient_id,'rmse'] = np.sqrt(mean_squared_error(y, y_pred))

patients.head()

In [ ]:
# for i in range(len(data)):
#     patient_id = data[i]["patient_id"]
#     with torch.no_grad():
#         img = data[i]['image'].unsqueeze(0).float().to(device)
#         latent_features = model.encode(img, return_partials=False).squeeze().cpu().numpy().tolist()
#         for j in range(len(latent_features)):
#             patients.loc[patients.Patient==patient_id,'l'+str(j)] = latent_features[j]

# patients.head(176)

In [ ]:
# plt.title("l4")
# mean = patients['l4'].mean()
# std = patients['l4'].std()
# sns.distplot(patients['l4'], bins=100);
# print(f'l4 mean: {mean}')
# print(f'l4 std: {std}')

In [ ]:
# plt.title("Gradients")
# mean = patients['gradient'].mean()
# std = patients['gradient'].std()
# sns.distplot(patients['gradient'], bins=100);
# print(f'Gradient mean: {mean}')
# print(f'Gradient std: {std}')

In [ ]:
# plt.title("Intercepts")
# mean = patients['intercept'].mean()
# std = patients['intercept'].std()
# # sns.distplot(patients['intercept'], bins=100);
# print(f'Intercept mean: {mean}')
# print(f'Intercept std: {std}')


In [ ]:
# plt.title("RMSEs")
# mean = patients['rmse'].mean()
# std = patients['rmse'].std()
# # sns.distplot(patients['rmse'], bins=100);
# print(f'RMSE mean: {mean}')
# print(f'RMSE std: {std}')

In [ ]:
corr_matrix = patients.corr()
sns.heatmap(corr_matrix)
corr_matrix

In [ ]:
corr_matrix.sort_values(by=['gradient'])[corr_matrix['gradient'].abs()>0.1]['gradient']

In [ ]:
corr_matrix.sort_values(by=['intercept'])[corr_matrix['intercept'].abs()>0.48]['intercept']

In [ ]:
corr_matrix.sort_values(by=['rmse'])[corr_matrix['rmse'].abs()>0.15]['rmse']

In [ ]:
x = patients['Class']
y = patients['intercept']
plt.title("intercept")
plt.scatter(x,y)

In [ ]:
# x = patients['FVC_base']
# y = patients['gradient']
# fig, ax = plt.subplots(figsize=(10,10));
# ax.set_title("gradient")
# sns.scatterplot(x,np.log(y),hue=patients['Class'], size=patients['Age'])

In [ ]:
# x = patients['Class']
# y = patients['rmse']
# plt.title("rmse")
# plt.scatter(x,y)

In [ ]:
num_features = ['Age', 'Class', 'Weeks_base', 'FVC_base',
       'Percent_base', 'FVC_nom_base', 'gradient', 'intercept']
target = ['rmse']

x = patients[num_features]
y = patients[target]
regr = linear_model.LinearRegression()
regr.fit(x,y)
plt.scatter(num_features, regr.coef_)
plt.xticks(rotation=90)
plt.title("RMSE")

In [ ]:
# num_features = ['Age', 'Class', 'Weeks_base', 'FVC_base',
#        'Percent_base', 'FVC_nom_base', 'gradient', 'rmse']
# target = ['intercept']

# x = patients[num_features]
# y = patients[target]
# regr = linear_model.LinearRegression()
# regr.fit(x,y)
# plt.scatter(num_features, regr.coef_)
# plt.xticks(rotation=90)
# plt.title("Intercept")

In [ ]:
# num_features = ['Age', 'Class', 'Weeks_base', 'FVC_base',
#        'Percent_base', 'FVC_nom_base', 'intercept', 'rmse']
# target = ['gradient']

# x = patients[num_features]
# y = patients[target]
# regr = linear_model.LinearRegression()
# regr.fit(x,y)
# plt.scatter(num_features, regr.coef_)
# plt.xticks(rotation=90)
# plt.title("Gradient")

In [ ]:
PatientID = train['Patient'].values
fvc_b = train.groupby('Patient').first()['FVC_base']
#fvc_b.values
age = train.groupby('PatientID').first()['Age']
age

In [ ]:
def model_fit(data, examine=True):
    n_patients = data['Patient'].nunique()
    FVC_obs = data['FVC'].values
    Weeks = data['Weeks'].values
    #X = train[['Weeks', 'Male', 'ExSmoker', 'CurrentlySmokes', 'Percent_base']].values
    PatientID = data['PatientID'].values
    patient_class = data['Class'].values
    FVC_b = data.groupby('PatientID').first()['FVC_base']
    w_b = data.groupby('PatientID').first()['Weeks_base']
    age = data.groupby('PatientID').first()['Age']
    # Small denotes 1 per patient, ie. not whole array of class values from all datapoints
    patient_class_small = data.groupby('PatientID').first()['Class']

    with pm.Model() as model:
        # create shared variables that can be changed later on
        FVC_obs_shared = pm.Data("FVC_obs_shared", FVC_obs)
        Weeks_shared = pm.Data('Weeks_shared', Weeks)
        PatientID_shared = pm.Data('PatientID_shared', PatientID)
        patient_class_shared = pm.Data('patient_class_shared', patient_class)
        FVC_b_shared = pm.Data('FVC_b_shared', FVC_b)
        w_b_shared= pm.Data('w_b_shared', w_b)
        age_shared = pm.Data('age_shared', age)
        patient_class_small_shared = pm.Data('patient_class_small_shared', patient_class_small)

        #mu_a_base = pm.Normal('mu_a_base', mu=0., sigma=100)
        #mu_a = FVC_b_shared + mu_a_base * w_b_shared
        mu_a = pm.Normal('mu_a', mu=1700., sigma=400)
        sigma_a = pm.HalfNormal('sigma_a', 1000.)
        
        
        alpha_s = pm.Normal('alpha_s', -7, sigma=2)
        beta_cs = pm.Normal('beta_cs', 0, sigma=2.5, shape=6)
        mu_b = alpha_s + beta_cs[patient_class_small_shared]*age_shared 
        sigma_b = pm.HalfNormal('sigma_b', 5.)
        
#         mu_b = pm.Normal('mu_b', mu=-4., sigma=1)
#         sigma_b = pm.HalfNormal('sigma_b', 5.)
        
        #mu_c = pm.Normal('mu_c', mu=0, sigma=100)
        #sigma_c = pm.HalfNormal('sigma_c', 100)

        a = pm.Normal('a', mu=mu_a, sigma=sigma_a, shape=n_patients)
        b = pm.Normal('b', mu=mu_b, sigma=sigma_b, shape=n_patients)
        #c = pm.Normal('c', mu=mu_c, sigma=sigma_c, shape=n_patients)

        # Model error
        #sigma = pm.HalfNormal('sigma', 150.)
        sigma = pm.HalfNormal('sigma', 150., shape=6)
        
        FVC_est = a[PatientID_shared] + b[PatientID_shared] * Weeks_shared

        # Data likelihood
        FVC_like = pm.Normal('FVC_like', mu=FVC_est,
                             sigma=sigma[patient_class_shared], observed=FVC_obs_shared)

        # Fitting the model
        trace = pm.sample(1000, tune=2000, target_accept=.9, init="adapt_diag")
        
    if examine:
        with model:
            pm.traceplot(trace);
            
    return model, trace

In [ ]:
# with model_a:
#     pm.traceplot(trace_a);

In [ ]:
def generate_template(data):
    pred_template = []
    for i, patient in enumerate(data['Patient'].unique()):
        df = pd.DataFrame(columns=['PatientID', 'Weeks', 'Patient', 'Class'])
        df['Weeks'] = np.arange(-12, 134)
        df['Patient'] = patient
        df['Class'] = data.loc[data['Patient']==patient]['Class'].max()
        pred_template.append(df)
    pred_template = pd.concat(pred_template, ignore_index=True)
    pred_template['PatientID'] = le_id.transform(pred_template['Patient'])
    return pred_template

In [ ]:
template_train_test = generate_template(test)
template_train_test.head()

In [ ]:
def model_predict(model, trace, template):
    with model:
        pm.set_data({
            "PatientID_shared": template['PatientID'].values.astype(int),
            "Weeks_shared": template['Weeks'].values.astype(int),
            "FVC_obs_shared": np.zeros(len(template)).astype(int),
            "patient_class_shared": template['Class'].values.astype(int),
        })
        post_pred = pm.sample_posterior_predictive(trace)
    df = pd.DataFrame(columns=['Patient', 'Weeks', 'FVC_pred', 'sigma'])
    df['Patient'] = le_id.inverse_transform(template['PatientID'])
    df['Weeks'] = template['Weeks']
    df['FVC_pred'] = post_pred['FVC_like'].T.mean(axis=1)
    df['sigma'] = (post_pred['FVC_like'].T.std(axis=1))
    df['FVC_inf'] = df['FVC_pred'] - df['sigma']
    df['FVC_sup'] = df['FVC_pred'] + df['sigma']
    df = pd.merge(df, train[['Patient', 'Weeks', 'FVC']], how='left', on=['Patient', 'Weeks'])
    #if exclude_test_patient_data_from_trainset:
    #df=df.append(train_raw[train_raw['Patient'].isin(test['Patient'].unique())][['Patient', 'Weeks', 'FVC']])
    df = df.rename(columns={'FVC': 'FVC_true'})
    return df

In [ ]:
def examine_predictions(data):
    n = (data['Patient'].nunique())+1
    f, axes = plt.subplots((n//3)+1, 3, figsize=(15, 5*((n//3)+1)))
    #for i, patient in enumerate(np.random.choice(data['Patient'].unique(), size=3, replace=False)):
    for i, patient in enumerate(data['Patient'].unique()):
        ax = axes[i//3, i%3]
        df = data[data['Patient'] == patient]
        x = df['Weeks']
        ax.set_title(patient)
        ax.plot(x, df['FVC_true'], 'o')
        ax.plot(x, df['FVC_pred'])
        ax = sns.regplot(x, df['FVC_true'], ax=ax, ci=None, line_kws={'color':'red'})
        ax.fill_between(x, df["FVC_inf"], df["FVC_sup"],alpha=0.5, color='#ffcd3c')
        ax.set_ylabel('FVC')
    axes[n//3,n%3].plot()


In [ ]:
def evaluate_predictions(df, use_only_last_3_measures=False, examine=False):
    if use_only_last_3_measures:
        y = df.dropna().groupby('Patient').tail(3)
    else:
        y = df.dropna()

    rmse = ((y['FVC_pred'] - y['FVC_true']) ** 2).mean() ** (1/2)
    mae = (y['FVC_pred'] - y['FVC_true']).abs()
    mae_mean = (np.sqrt((y['FVC_pred'] - y['FVC_true']) ** 2)).mean()
    mae_sd = (np.sqrt((y['FVC_pred'] - y['FVC_true']) ** 2)).mean()
    mae_max = (np.sqrt((y['FVC_pred'] - y['FVC_true']) ** 2)).mean()
    #print(f'RMSE (mean): {rmse:.1f} ml')
    #print(f'MAE (mean): {rmse:.1f} ml')
    sigma_c = y['sigma'].values
    sigma_c[sigma_c < 70] = 70
    delta = (y['FVC_pred'] - y['FVC_true']).abs()
    delta[delta > 1000] = 1000
    lll = - np.sqrt(2) * delta / sigma_c - np.log(np.sqrt(2) * sigma_c)

    #print(f'Laplace Log Likelihood: {lll.mean():.4f}')
    y['sigma_c'] = y['sigma']
    y['sigma_c'].values[y['sigma_c'].values < 70] = 70
    y['delta_c'] = (y['FVC_pred'] - y['FVC_true']).abs()
    y['delta_c'].values[y['delta_c'].values > 1000] = 1000
    y['main_loss'] = y['delta_c']/y['sigma_c']
    y['lll'] = - np.sqrt(2) * y['delta_c'] / y['sigma_c'] - np.log(np.sqrt(2) * y['sigma_c'])
    patient_llls = y.groupby('Patient')['lll'].mean()
    if examine:
        #plt.hist(y['main_loss'], bins=100)
        plt.hist(patient_llls, bins=100);
    #bad_patients = y[y['main_loss'] > 2.5]
    #bad_patients
    #bad_patients['Patient'].unique()

    #plt.scatter(delta, sigma_c);
    #plt.xlabel("delta");
    #plt.ylabel("sigma_c");
    return lll.mean(), patient_llls

In [ ]:
def evaluation_cycle(train, valid, examine_trace=True, examine_preds=True):
    # Fit model
    print("Fit model ...")
    model,trace = model_fit(train, examine=examine_trace)

    # Model predictions with training data
    print("Examine true vs predictions for training data ...")
    template_train = generate_template(train)
    template_train.head()
    pred_train = model_predict(model,trace,template_train)
    if examine_preds:
        examine_predictions(pred_train)
    lll_train, train_patient_llls = evaluate_predictions(pred_train)

    # Model predictions with validation data (unseen)
    if valid is not None:
        print("Examine true vs predictions for validation data ...")
        template_valid = generate_template(valid)
        pred_valid = model_predict(model,trace,template_valid)
        if examine_preds:
            examine_predictions(pred_valid)
        lll_valid, valid_patient_llls = evaluate_predictions(pred_valid)
    return pred_train, pred_valid, lll_train, lll_valid, train_patient_llls, valid_patient_llls

In [ ]:
#Evaluate model

lll_trains = []
lll_valids = []

fold = 0
kfold = KFold(3, shuffle=True, random_state=1)
all_patients = train['Patient'].unique()
# Fold
# for train_index, validation_index in kfold.split(all_patients):
#     validation_patients = all_patients[validation_index]
#     fold += 1

# Random
for fold in range(1):
    validation_patients = np.random.choice(all_patients, size=30, replace=False)
    
    examine_lll = True
    
    df_valid = train[train['Patient'].isin(validation_patients)]
    df_train = train[~train['Patient'].isin(validation_patients)]

    df_valid_first_readings = df_valid.groupby('Patient').head(1)
    df_train = pd.concat([df_train, df_valid_first_readings], axis=0, ignore_index=True)

    print(f'Fold: {fold}')
    pred_train, pred_valid, lll_train, lll_valid, train_patient_llls, valid_patient_llls = evaluation_cycle(df_train, df_valid, examine_trace=True, examine_preds=False)

    print(f'Laplace Log Likelihoods for fold: {fold}')
    print(f'Training:     {lll_train:.4f}')
    print(f'Validation:   {lll_valid:.4f}')
    print("")
        
    train_worst = train_patient_llls.nsmallest(10).index
    valid_worst = valid_patient_llls.nsmallest(10).index
    
    print(f'Training - worst:   {*train_worst,}')
    print(f'Validation - worst: {*valid_worst,}')
    print("")
    pred_train_worst = pred_train[pred_train['Patient'].isin(train_worst)]
    pred_valid_worst = pred_valid[pred_valid['Patient'].isin(valid_worst)]
    
    examine_predictions(pred_train_worst)
    examine_predictions(pred_valid_worst)
    
    lll_trains.append(lll_train)
    lll_valids.append(lll_valid)
    
    evaluate_predictions(pred_train, use_only_last_3_measures=True, examine=examine_lll)
    evaluate_predictions(pred_valid, use_only_last_3_measures=True, examine=examine_lll)

print(f'Training LLLs:   {*lll_trains,}')
print(f'Validation LLLs: {*lll_valids,}')
print("")

lll_trains_array = np.array(lll_trains)
lll_valids_array = np.array(lll_valids)

print(f'Training LLLs Mean:   {lll_trains_array.mean():.4f}')
print(f'Training LLLs STD:    {lll_trains_array.std():.4f}')
print("")
print(f'Validation LLLs Mean: {lll_valids_array.mean():.4f}')
print(f'Validation LLLs STD:  {lll_valids_array.std():.4f}')



In [ ]:
# Fit model using all of training data
print("Fit model ...")
model,trace = model_fit(train, examine=False)
print("")

# Make predictions for test data
print("Make predictions for test data ...")
template_test = generate_template(test)
template_test.head()
pred_test = model_predict(model,trace,template_test)
#examine_predictions(pred_test)

# Prepare final submission
final = pd.DataFrame(columns=['Patient_Week', 'FVC', 'Confidence'])
final['Patient_Week'] = pred_test['Patient'] + '_' + pred_test['Weeks'].astype(str)
final['FVC'] = pred_test['FVC_pred']
final['Confidence'] = pred_test['sigma']
final.head()
final.to_csv('submission.csv', index=False)
print(final.shape)
final.head()

In [ ]:
# pred_template = []
# for p in test['Patient'].unique():
#     df = pd.DataFrame(columns=['PatientID', 'Weeks'])
#     df['Weeks'] = np.arange(-12, 134)
#     df['Patient'] = p
#     pred_template.append(df)
# pred_template = pd.concat(pred_template, ignore_index=True)
# pred_template['PatientID'] = le_id.transform(pred_template['Patient'])

# with model_a:
#     pm.set_data({
#         "PatientID_shared": pred_template['PatientID'].values.astype(int),
#         "Weeks_shared": pred_template['Weeks'].values.astype(int),
#         "FVC_obs_shared": np.zeros(len(pred_template)).astype(int),
#     })
#     post_pred = pm.sample_posterior_predictive(trace_a)

In [ ]:
# df = pd.DataFrame(columns=['Patient', 'Weeks', 'Patient_Week', 'FVC', 'Confidence'])
# df['Patient'] = pred_template['Patient']
# df['Weeks'] = pred_template['Weeks']
# df['Patient_Week'] = df['Patient'] + '_' + df['Weeks'].astype(str)
# df['FVC'] = post_pred['FVC_like'].T.mean(axis=1)
# df['Confidence'] = post_pred['FVC_like'].T.std(axis=1)
# final = df[['Patient_Week', 'FVC', 'Confidence']]
# final.to_csv('submission.csv', index=False)
# print(final.shape)
# final.head()